###mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###libraries


In [ ]:
!pip install langchain
!pip install openai
!pip install langchain-experimental
!pip install langchainhub

In [ ]:
from langchain import OpenAI, SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain import OpenAI, ConversationChain

In [ ]:
import os
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field

###LLM



In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'Your Token'
llm = OpenAI(temperature=0)


In [ ]:
import sqlite3

db = SQLDatabase.from_uri("sqlite:///Benchmark/Elastic_benchmark_issues_unstructured.db")
db_cfg = SQLDatabase.from_uri("sqlite:///Benchmark/Elastic_benchmark_issues_structured.db")

def retriever(query):
  if(cfg):
    print('cfg')
    db_chain_cfg = SQLDatabaseChain.from_llm(llm, db_cfg, verbose=True,use_query_checker=True, top_k=40)#,memory=memory)
    return db_chain_cfg.run(query)
  else:
      db_chain_normal = SQLDatabaseChain.from_llm(llm, db, verbose=True,use_query_checker=True, top_k=40)#,memory=memory)
      return db_chain_normal.run(query)

##Query

In [ ]:
examples = [
    {
        "input": "Find unresolved issues with no activity in the last 6 months",
        "output": "Select issue numbers of open issues with last activity date older than 6 months ago."
    },
    {
        "input": "Suggest existing labels to tag issue 18608?",
        "output": "List all existing labels and find suitable one for issue 18608 based on its content"
    },
    {
        "input": "Is issue 18102 and 18669 similar?",
        "output": "Compare the exceptions, stack traces, and descriptions of issues 18102 and 18669 to determine similarity."
    },
    {
        "input": "Are there any issues similar to issue 18669?",
        "output": "Identify issues with exceptions similar to those in issue 18669."
    },
    {
        "input": "How many times did the internal grinder tests fail in issue 17852?",
        "output": "Extract the number of internal grinder test failures mentioned in issue 17852."
    }
]


In [ ]:
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", """You need to extract query information from a database of issues.
     Your task is to rephrase the question to make it more concise and direct, without altering its core intent or specificity.
     Questions are of four intents: Yes/no, fact, summarization and list.
     1. Yes/No: Change 'is/are/have there issues' to  check if there any issues with provided condition
     2. List: Change 'Find issues' to "List issue numbers" with provided condition
     3. Summarization: Summarize the contents from issue title, exceptions, body and labels
     4. Fact: Extract the fact
     Retain keywords and intent as it is.

     Follow these examples:"""),
    # Few shot examples
    few_shot_prompt,
    # New question
    ("user", "{question}"),
])
question_gen = prompt | ChatOpenAI(temperature=0) | StrOutputParser()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


##Process Questions

In [ ]:
import time
def process_questions(questions):
    answers = []

    for i, q in enumerate(questions):
        #try:
            if(change_query):
              print('query')
              q=question_gen.invoke({"question": q})


            if(MT):
              if(COVE):
                print('mt,cove')
                init_response=Cove(q)
              else:
                init_response=retriever(q)
              answer=MT_answer(q,init_response)

            elif(COVE):
              print('Cove')
              answer=Cove(q)
            else:
              answer=retriever(q)

            answers.append(answer)
            print(f'-----------Final-------------: {answer}\n')

        #except Exception as e:
                #print(f"An error occurred while processing question {i+1}: {e}")
                #answers.append(None)  # Append None if an error occurs or question is skipped

    return answers

In [ ]:
import csv

def write_to_csv(file_path, questions, answers):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Question', 'Answer'])
        for question, answer in zip(questions, answers):
            writer.writerow([question, answer])


##Configure

In [ ]:
name=''

cfg=True
change_query=True
COVE=True
MT=True

#process_questions(['Identify one team with highest open issues over 8 months'])

if(cfg):
  name+='_CFG'
if(change_query):
  name+='_Query'
if(COVE):
  name+='_COVE'
if(MT):
  name+='_MT'

if(cfg and change_query and COVE and MT):
  name='_CIMBUR'  ## or CHIME. CIMBUR is the old name of CHIME

if not (cfg or change_query or COVE or MT):
    name = '_LLM'

print(name)


#result_df=pd.read_csv('Result/Correctness.csv')
#count_df=pd.read_csv('Correct_Count.csv')

_CIMBUR


##Cove

In [ ]:
llm = ChatOpenAI(temperature=0) #, model_name="gpt-4"

input_variables = ["query"]
base_response_output_key = "base_response"
base_response_template = """Question: {query} Answer:"""
base_repsonse_prompt_template = PromptTemplate(
    input_variables=input_variables, template=base_response_template
)
base_response_chain = LLMChain(
    llm=llm, prompt=base_repsonse_prompt_template,output_key=base_response_output_key)




In [ ]:
def verify(query):

  input_variables = ["query"]
  base_response_output_key = "base_response"
  base_response_template = """Question: {query} Answer:"""
  base_repsonse_prompt_template = PromptTemplate(
      input_variables=input_variables, template=base_response_template
  )
  base_response_chain = LLMChain(
      llm=llm, prompt=base_repsonse_prompt_template,output_key=base_response_output_key)


  if(cfg):
    db = SQLDatabase.from_uri("sqlite:///Benchmark/Elastic_benchmark_issues_structured.db")
  else:
     db = SQLDatabase.from_uri("sqlite:///Benchmark/Elastic_benchmark_issues_unstructured.db")

  db_chain_normal = SQLDatabaseChain.from_llm(llm, db, verbose=True,use_query_checker=True,top_k=40,output_key=base_response_output_key)
  def normal_retriever(query):
      return db_chain_normal.run(query)


  plan_verifications_template = """
  Given the below Question and answer, generate a series of verification questions that test the factual claims in the original baseline response.
  For example if part of a longform model response contains the statement “The 2 exception types found in the issue report are java.io.EOFException, AssertionError”, then one possible
  verification question to check those data could be “can java.io.EOFException be found in existing issue reports. If there is issue number
  and asking about this issue only, focus on it.”

  Question: {query}
  Answer: {base_response}

  <fact in query and passage>, <verification question, generated by combining the query and the fact>

  {format_instructions}
  """

  class PlanVerificationsOutput(BaseModel):
      query: str = Field(description="The user's query", default="")
      base_response: str = Field(description="The response to the user's query", default="")
      facts_and_verification_questions: dict[str, str] = Field(
          description="Facts (as the dictionary keys) extracted from the response and verification questions related to the query (as the dictionary values)"
      , default="")

  try:
    plan_verifications_output_parser = PydanticOutputParser(
        pydantic_object=PlanVerificationsOutput
    )
    plan_verifications_prompt_template = PromptTemplate(
        input_variables=input_variables + [base_response_output_key],
        template=plan_verifications_template,
        partial_variables={
            "format_instructions": plan_verifications_output_parser.get_format_instructions()
        },
    )
    plan_verifications_chain = LLMChain(
        llm=llm,
        prompt=plan_verifications_prompt_template,
        output_key="output",
        output_parser=plan_verifications_output_parser,
    )


    answer_and_plan_verification = SequentialChain(
        chains=[db_chain_normal, plan_verifications_chain],
        input_variables=["query"],
        output_variables=["output"],
        verbose=True)

    intermediate_result = answer_and_plan_verification.run(query)

    claimed_facts = list(intermediate_result.facts_and_verification_questions.keys())
    verification_questions = list(
        intermediate_result.facts_and_verification_questions.values()
    )
    verify_results_str = ""
    verify_input_variables = ["question"]
    verify_output_key = "answer"
    verify_template = """{question}"""

    verify_prompt_template = PromptTemplate(
        input_variables=verify_input_variables, template=verify_template
    )

    #verify_chain = LLMChain(
    #    llm=llm, prompt=verify_prompt_template, output_key=verify_output_key)


    verify_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True,use_query_checker=True,output_key=verify_output_key)#,memory=memory)


    # Answering verification questions independently
    for i in range(len(verification_questions)):
        claimed_fact = claimed_facts[i]
        question = verification_questions[i]
        answer = verify_chain.run(question)
        answer = answer.lstrip("\n")
        verify_results_str += f"Question: {question}\nAnswer: {answer}\n\n"
    return intermediate_result, verify_results_str
  except Exception as e:
    print(e)
    return None, ''

#intermediate_result, verify_results_str= verify('what is the exception in issue 18151',False)


In [ ]:
def Cove(query):
  try:
    intermediate_result,verify_results_str=verify(query)
    if(intermediate_result==None):
      return ''
    final_response_input_variables = ["query", "base_response", "verify_results"]
    final_response_template = """Given the ORIGINAL_QUESTION and the ORIGINAL_RESPONSE,
    revise the ORIGINAL_RESPONSE (if applicable) such that it is consistent with information in VERIFIED_SOURCE as answer for ORIGINAL_QUESTION.
    Only keep consistent information.

    <ORIGINAL_QUESTION>
    {query}

    <ORIGINAL_RESPONSE>
    {base_response}

    <VERIFIED_SOURCE>
    {verify_results}

    Final response:
    """
    final_response_prompt_template = PromptTemplate(
        input_variables=final_response_input_variables,
        template=final_response_template,
    )

    final_response_chain = LLMChain(llm=llm, prompt=final_response_prompt_template)


    final_response = final_response_chain.run(
        query=intermediate_result.query,
        base_response=intermediate_result.base_response,
        # verify_results=verify_results_str,
        # Update 2023-10-22: use results from internet search
        verify_results=verify_results_str,
    )
    print(final_response)
    return final_response
  except Exception as e:
    return ''


##MT

In [ ]:
from langchain.chat_models import ChatOpenAI

def mutate_query(original_query, n_mutations=3):

        # Prompt for the LLM to generate mutations
        template = """
        Generate 3 different ways to ask the following question, keeping the semantic meaning the same:\n\n'{input}'\n\nMutations:
        text: {input}
        """
        # Initialize the ChatOpenAI model

        llm = OpenAI(temperature=0)
        prompt_template = PromptTemplate.from_template(template=template)
        chain = LLMChain(llm=llm, prompt=prompt_template)
        response=chain.predict(input=original_query)
        print(response)

        # Extract and return the mutated queries
        # Assuming the response format aligns with your expectations

         # Assuming the response is a string, split and format it appropriately
        mutated_queries = response.split("\n")  # Adjust this based on the actual format of 'response'

        # Filter out empty lines or irrelevant parts
        mutated_queries = [query.strip() for query in mutated_queries if query.strip() and not query.strip().startswith("Mutations")]

        return mutated_queries

In [ ]:

def MT_answer(original_query,init_response):
  try:
    mutated_queries = mutate_query(original_query)
    print(mutated_queries)

    verify_results_str =""
    for q in mutated_queries:
      a=retriever(q)
      verify_results_str+= f"Question: {q}\nAnswer: {a}\n\n"

    query = original_query
    base_response = init_response


    print('BASE--------')
    print(base_response)
    print('verify-----')
    print(verify_results_str)


    # Final response prompt template
    final_response_input_variables = ["query", "base_response", "verify_results"]
    final_response_template = """Given the ORIGINAL_QUESTION and the ORIGINAL_RESPONSE,
    revise the ORIGINAL_RESPONSE (if applicable) such that it is consistent with information in VERIFIED_SOURCE as answer for ORIGINAL_QUESTION.
    Only keep consistent information.

    <ORIGINAL_QUESTION>
    {query}

    <ORIGINAL_RESPONSE>
    {base_response}

    <VERIFIED_SOURCE>
    {verify_results}

    Final response:
    """

    final_response_prompt_template = PromptTemplate(
        input_variables=final_response_input_variables,
        template=final_response_template,
    )

    final_response_chain = LLMChain(llm=llm, prompt=final_response_prompt_template)

    # Running the chain to get the final response
    final_response = final_response_chain.run(
        query=query,
        base_response=base_response,
        verify_results=verify_results_str,
    )

    print('final-----')
    print("Final Response:", final_response)
    return final_response
  except Exception as e:
    return ''


##Backlog

###y/n

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T5.csv')

single_q = df.loc[df['T5-YNQ'].notnull(), 'T5-YNQ']
single_a=process_questions(single_q)

df[f'T5-YNA{name}'] = ''
df.loc[:len(single_a)-1, f'T5-YNA{name}'] = single_a

df.to_csv('Benchmark/E-T5.csv', index=False)

# Get actual and expected answers
actual_answers = df.loc[df[f'T5-YNA{name}'].notnull(), f'T5-YNA{name}']
expected_answers = df.loc[df['T5-YNA'].notnull(), 'T5-YNA']

# Call the function
results, correct, correctness = classify_and_evaluate(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T5-YN{name}_R'] = ''
df[f'T5-YN{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T5.csv', index=False)

result_df[f'T5-YN{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T5-YN{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###sum

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T5.csv')

single_q = df.loc[df['T5-SQ'].notnull(), 'T5-SQ']
single_a=process_questions(single_q)

df[f'T5-SA{name}'] = ''
df.loc[:len(single_a)-1, f'T5-SA{name}'] = single_a

df.to_csv('Benchmark/E-T5.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T5-SA'].notnull(), 'T5-SA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T5-S{name}_R'] = ''
df[f'T5-S{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T5.csv', index=False)

result_df[f'T5-S{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T5-S{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###fact

In [ ]:

import pandas as pd
df = pd.read_csv('Benchmark/E-T5.csv')

single_q = df.loc[df['T5-FQ'].notnull(), 'T5-FQ']
single_a=process_questions(single_q)

df[f'T5-FA{name}'] = ''
df.loc[:len(single_a)-1, f'T5-FA{name}'] = single_a

df.to_csv('Benchmark/E-T5.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T5-FA'].notnull(), 'T5-FA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)


##Label

###y/n

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T4.csv')

single_q = df.loc[df['T4-YNQ'].notnull(), 'T4-YNQ']
single_a=process_questions(single_q)

df[f'T4-YNA{name}'] = ''
df.loc[:len(single_a)-1, f'T4-YNA{name}'] = single_a

df.to_csv('Benchmark/E-T4.csv', index=False)

# Get actual and expected answers
actual_answers = df.loc[df[f'T4-YNA{name}'].notnull(), f'T4-YNA{name}']
expected_answers = df.loc[df['T4-YNA'].notnull(), 'T4-YNA']

# Call the function
results, correct, correctness = classify_and_evaluate(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T4-YN{name}_R'] = ''
df[f'T4-YN{name}_R'][:len(results)] = results


df.to_csv('Benchmark/E-T4.csv', index=False)

result_df[f'T4-YN{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T4-YN{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)


###sum

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T4.csv')

single_q = df.loc[df['T4-SQ'].notnull(), 'T4-SQ']
single_a=process_questions(single_q)

df[f'T4-SA{name}'] = ''
df.loc[:len(single_a)-1, f'T4-SA{name}'] = single_a

df.to_csv('Benchmark/E-T4.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T4-SA'].notnull(), 'T4-SA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T4-S{name}_R'] = ''
df[f'T4-S{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T4.csv', index=False)

result_df[f'T4-S{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T4-S{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###fact

In [ ]:

import pandas as pd
df = pd.read_csv('Benchmark/E-T4.csv')

single_q = df.loc[df['T4-FQ'].notnull(), 'T4-FQ']
single_a=process_questions(single_q)

df[f'T4-FA{name}'] = ''
df.loc[:len(single_a)-1, f'T4-FA{name}'] = single_a

df.to_csv('Benchmark/E-T4.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T4-FA'].notnull(), 'T4-FA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T4-FA{name}_R'] = ''
df[f'T4-FA{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T4.csv', index=False)

result_df[f'T4-F{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T4-F{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

##Summary


###sum

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T3.csv')

single_q = df.loc[df['T3-SQ'].notnull(), 'T3-SQ']
single_a=process_questions(single_q)

df[f'T3-SA{name}'] = ''
df.loc[:len(single_a)-1, f'T3-SA{name}'] = single_a

df.to_csv('Benchmark/E-T3.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T3-SA'].notnull(), 'T3-SA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T3-S{name}_R'] = ''
df[f'T3-S{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T3.csv', index=False)

result_df[f'T3-S{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T3-S{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###fact

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T3.csv')

single_q = df.loc[df['T3-FQ'].notnull(), 'T3-FQ']
single_a=process_questions(single_q)

df[f'T3-FA{name}'] = ''
df.loc[:len(single_a)-1, f'T3-FA{name}'] = single_a

df.to_csv('Benchmark/E-T3.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T3-FA'].notnull(), 'T3-FA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T3-FA{name}_R'] = ''
df[f'T3-FA{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T3.csv', index=False)


result_df[f'T3-F{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T3-F{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

##Trend

###y/n

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T2.csv')

single_q = df.loc[df['T2-YNQ'].notnull(), 'T2-YNQ']
single_a=process_questions(single_q)

df[f'T2-YNA{name}'] = ''
df.loc[:len(single_a)-1, f'T2-YNA{name}'] = single_a

df.to_csv('Benchmark/E-T2.csv', index=False)

# Get actual and expected answers
actual_answers = df.loc[df[f'T2-YNA{name}'].notnull(), f'T2-YNA{name}']
expected_answers = df.loc[df['T2-YNA'].notnull(), 'T2-YNA']

# Call the function
results, correct, correctness = classify_and_evaluate(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T2-YN{name}_R'] = ''
df[f'T2-YN{name}_R'][:len(results)] = results


df.to_csv('Benchmark/E-T2.csv', index=False)

result_df[f'T2-YN{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T2-YN{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)


###sum

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T2.csv')

single_q = df.loc[df['T2-SQ'].notnull(), 'T2-SQ']
single_a=process_questions(single_q)

df[f'T2-SA{name}'] = ''
df.loc[:len(single_a)-1, f'T2-SA{name}'] = single_a

df.to_csv('Benchmark/E-T2.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T2-SA'].notnull(), 'T2-SA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T2-S{name}_R'] = ''
df[f'T2-S{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T2.csv', index=False)


result_df[f'T2-S{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T2-S{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###fact

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T2.csv')

single_q = df.loc[df['T2-FQ'].notnull(), 'T2-FQ']
single_a=process_questions(single_q)

df[f'T2-FA{name}'] = ''
df.loc[:len(single_a)-1, f'T2-FA{name}'] = single_a

df.to_csv('Benchmark/E-T2.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T2-FA'].notnull(), 'T2-FA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T2-FA{name}_R'] = ''
df[f'T2-FA{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T2.csv', index=False)


result_df[f'T2-F{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T2-F{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

##Similar

###y/n

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T1M.csv')

single_q = df.loc[df['T1M-YNQ'].notnull(), 'T1M-YNQ']
single_a=process_questions(single_q)

df[f'T1M-YNA{name}'] = ''
df.loc[:len(single_a)-1, f'T1M-YNA{name}'] = single_a

df.to_csv('Benchmark/E-T1M.csv', index=False)

# Get actual and expected answers
actual_answers = df.loc[df[f'T1M-YNA{name}'].notnull(), f'T1M-YNA{name}']
expected_answers = df.loc[df['T1M-YNA'].notnull(), 'T1M-YNA']

# Call the function
results, correct, correctness = classify_and_evaluate(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T1M-YN{name}_R'] = ''
df[f'T1M-YN{name}_R'][:len(results)] = results


df.to_csv('Benchmark/E-T1M.csv', index=False)

result_df[f'T1M-YN{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T1M-YN{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)


###sum

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T1M.csv')

single_q = df.loc[df['T1M-SQ'].notnull(), 'T1M-SQ']
single_a=process_questions(single_q)

df[f'T1M-SA{name}'] = ''
df.loc[:len(single_a)-1, f'T1M-SA{name}'] = single_a

df.to_csv('Benchmark/E-T1M.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T1M-SA'].notnull(), 'T1M-SA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T1M-S{name}_R'] = ''
df[f'T1M-S{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T1M.csv', index=False)

result_df[f'T1M-S{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T1M-S{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###fact

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T1M.csv')

single_q = df.loc[df['T1M-FQ'].notnull(), 'T1M-FQ']
single_a=process_questions(single_q)

df[f'T1M-FA{name}'] = ''
df.loc[:len(single_a)-1, f'T1M-FA{name}'] = single_a

df.to_csv('Benchmark/E-T1M.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T1M-FA'].notnull(), 'T1M-FA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T1M-FA{name}_R'] = ''
df[f'T1M-FA{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T1M.csv', index=False)



result_df[f'T1M-F{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T1M-F{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

##Single

###y/n

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T1S.csv')

single_q = df.loc[df['T1S-YNQ'].notnull(), 'T1S-YNQ']
single_a=process_questions(single_q)

df[f'T1S-YNA{name}'] = ''
df.loc[:len(single_a)-1, f'T1S-YNA{name}'] = single_a

df.to_csv('Benchmark/E-T1S.csv', index=False)

# Get actual and expected answers
actual_answers = df.loc[df[f'T1S-YNA{name}'].notnull(), f'T1S-YNA{name}']
expected_answers = df.loc[df['T1S-YNA'].notnull(), 'T1S-YNA']

# Call the function
results, correct, correctness = classify_and_evaluate(actual_answers, expected_answers)
print("Correctness:", correctness)




In [ ]:
df[f'T1S-YN{name}_R'] = ''
df[f'T1S-YN{name}_R'][:len(results)] = results


df.to_csv('Benchmark/E-T1S.csv', index=False)

result_df[f'T1S-YN{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T1S-YN{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###sum

In [ ]:
name=''

cfg=True
change_query=True
COVE=True
MT=True

#process_questions(['Identify one team with highest open issues over 8 months'])

if(cfg):
  name+='_CFG'
if(change_query):
  name+='_Query'
if(COVE):
  name+='_COVE'
if(MT):
  name+='_MT'

if(cfg and change_query and COVE and MT):
  name='_CIMBUR'

if not (cfg or change_query or COVE or MT):
    name = '_LLM'

print(name)


_CIMBUR


In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T1S.csv')

single_q = df.loc[df['T1S-SQ'].notnull(), 'T1S-SQ']
single_a=process_questions(single_q)

df[f'T1S-SA{name}'] = ''
df.loc[:len(single_a)-1, f'T1S-SA{name}'] = single_a

df.to_csv('Benchmark/E-T1S.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T1S-SA'].notnull(), 'T1S-SA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)

df[f'T1S-S{name}_R'] = ''
df[f'T1S-S{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T1S.csv', index=False)

result_df[f'T1S-S{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T1S-S{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

###fact

In [ ]:
import pandas as pd
df = pd.read_csv('Benchmark/E-T1S-F.csv')

single_q = df.loc[df['T1S-FQ'].notnull(), 'T1S-FQ']
single_a=process_questions(single_q)

df[f'T1S-FA{name}'] = ''
df.loc[:len(single_a)-1, f'T1S-FA{name}'] = single_a

df.to_csv('Benchmark/E-T1S-F.csv', index=False)

# Get actual and expected answers
actual_answers = single_a
expected_answers = df.loc[df['T1S-FA'].notnull(), 'T1S-FA']

# Call the function
results, correct, correctness = evaluate_summary(actual_answers, expected_answers)
print("Correctness:", correctness)



In [ ]:
df[f'T1S-FA{name}_R'] = ''
df[f'T1S-FA{name}_R'][:len(results)] = results

df.to_csv('Benchmark/E-T1S-F.csv', index=False)


result_df[f'T1S-F{name}']=[correctness]
result_df.to_csv('Result/Correctness.csv', index=False)

count_df[f'T1S-F{name}']=[correct]
count_df.to_csv('Correct_Count.csv', index=False)

<ipython-input-331-8182bde49aa6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'T1S-FA{name}_R'][:len(results)] = results


##Evaluation

###y/n

In [ ]:
import pandas as pd
from transformers import pipeline
classifier = pipeline("zero-shot-classification")

In [ ]:


def classify_and_evaluate(actual_answers, expected_answers):
    # Initialize the zero-shot classification pipeline


    # Define classes
    classes = ["Yes", "No"]

    correct = 0
    total = 0
    results = []

    for actual, expected in zip(actual_answers, expected_answers):
        print(actual)
        print(expected)
        total += 1
        result = classifier(actual, candidate_labels=classes, hypothesis_template="This statement implies: {}.")
        if result['labels'][0].lower() == expected.lower():
            correct += 1
            results.append(1)
            print(1)
        else:
            results.append(0)
            print(0)

    print('Correct '+ str(correct))
    correctness = correct / total
    return results, correct, correctness



###fact

In [ ]:
pip install sentence-transformers

In [ ]:
import re
def extract_numbers(text):
    return re.findall(r'\b\d+\b', text)

def clean_text(text):
    text = re.sub(r'[^\w\s:]', '', text)
    text = text.strip().replace('\n', '').replace('"', '').replace("'", '')
    return text

def containsOnlyNumbers(text):


  text = re.sub(r'[^\w\s]', '', text)
  text = text.replace('\n', '').replace('and', '')

  all_numbers = ' '.join(extract_numbers(text))

  print('=------erar-----')
  print(text)
  print(all_numbers)

  if(text==all_numbers):
        print(text)
        return True

  return False



def evaluate_fact(actual_answers, expected_answers):

    results = []
    correct=0
    # Evaluate correctness through semantic similarity
    for actual, expected in  zip(actual_answers, expected_answers):

        if(actual==None and expected==None):
          return
        if(actual==None):
          actual='None'

        expected=expected.strip()
        actual=actual.strip()
        similarity=0

        if(expected.isdigit() or len(expected.split())==1):
          print('first case')

          actual_s=set(clean_text(actual).split())
          expected_s=set(clean_text(expected).split())

          actual_s = set(word.lower() for word in actual_s)
          expected_s = set(word.lower() for word in expected_s)

          intersection = actual_s.intersection(expected_s)

          print(actual)
          print(expected)

          if intersection ==  set(expected.split()) or intersection:
            print('intersection')
            print(intersection)
            similarity=1


        elif containsOnlyNumbers(expected):

              print('second case')


              expected_num=sorted(extract_numbers(expected))
              expected_s = ' '.join(expected_num)
              min_limit=min(expected_num)
              max_limit=max(expected_num)

              actual_num=sorted(extract_numbers(actual))
              if(len(actual_num)!=0):

                actual_num = [num for num in actual_num if num > min_limit or num<=max_limit]
                actual_s = ' '.join(actual_num)
                print(actual_s)
                actual_embedding = model.encode(actual_s, convert_to_tensor=True)
                expected_embedding = model.encode(expected_s, convert_to_tensor=True)
                similarity = util.pytorch_cos_sim(actual_embedding, expected_embedding).item()



              print(expected_s)
              print(similarity)

        else:
              print('third case')
              actual_embedding = model.encode(actual, convert_to_tensor=True)
              expected_embedding = model.encode(expected, convert_to_tensor=True)
              similarity = util.pytorch_cos_sim(actual_embedding, expected_embedding).item()

              print(actual)
              print(expected)



        threshold = 0.7
        print(similarity)
        is_correct = 1 if similarity >= threshold else 0
        print(is_correct)
        if(is_correct==1):
          correct=correct+1
        results.append(is_correct)


    print('Correct '+ str(correct))
    correctness=correct/len(results)
    return results, correct, correctness

###sum

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:

def evaluate_summary(actual_answers, expected_answers):

    results = []
    correct=0


    # Evaluate correctness through semantic similarity
    for actual, expected in  zip(actual_answers, expected_answers):
        if(actual==None):
          actual='None'
        actual_embedding = model.encode(actual, convert_to_tensor=True)
        print(actual)
        expected_embedding = model.encode(expected, convert_to_tensor=True)
        print(expected)
        similarity = util.pytorch_cos_sim(actual_embedding, expected_embedding).item()

        # Assuming a threshold for "correctness" - this threshold is arbitrary and can be adjusted
        threshold = 0.7
        print(similarity)
        print(round(similarity))
        is_correct = 1 if similarity >= threshold else 0
        print(is_correct)
        if(is_correct==1):
          correct=correct+1

        results.append(is_correct)
    print('Correct '+ str(correct))
    correctness=correct/len(results)
    return results, correct, correctness